Today we're going to build something with immediate value!

In the folder me there is a file linkedin.pdf - a pdf download from my linkedin profile.

there is also a summary.txt file.

Not using tools yet in this example.

In [1]:
# Package imports
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text


In [4]:
print(linkedin)

   
Contact
8374496874 (Work)
kumarrav3053@hotmail.com
www.linkedin.com/in/ravi-kumar-
pidintla-152802b2 (LinkedIn)
Top Skills
Generative AI
Conflict Resolution
Agile Methodologies
Languages
Telugu (Native or Bilingual)
Bengali (Limited Working)
Italian (Elementary)
English (Full Professional)
Malayalam (Elementary)
Hindi (Full Professional)
Certifications
Generative AI for Everyone
Ravi Kumar Pidintla
Engineering Manager | SRE & Infra Automation | Building Reliable,
Observable, Data-Driven Systems
Andhra Pradesh, India
Summary
I'm an Engineering Manager with a strong foundation in Site
Reliability Engineering (SRE), infrastructure automation, and data-
driven platform operations. I lead teams focused on building reliable,
observable, and scalable systems that enable confident, high-
velocity delivery.
I specialize in:
- Leading high-impact SRE and platform teams to improve
availability, incident response, and operational maturity.
- Architecting automated infrastructure using Python, 

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

In [6]:
name = "Ravi"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [8]:
system_prompt

"You are acting as Ravi. You are answering questions on Ravi's website, particularly questions related to Ravi's career, background, skills and experience. Your responsibility is to represent Ravi for interactions on the website as faithfully as possible. You are given a summary of Ravi's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Ravi, I'm an Engineering Manager with a strong foundation in Site Reliability Engineering (SRE), infrastructure automation, and data-driven platform operations. My work is focused on building reliable, scalable, and observable systems that empower engineering teams to move fast without compromising on stability or insight.\n\nOver the years, I’ve led cross-functional SRE and platform teams, designing and delivering automation that improves incident respons

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

In [10]:
# gr.ChatInterface(chat, type="messages").launch()

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator the decides whether a response to a question is accpetable. You are provided with a converstaion between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing a role of {name} and it representing {name} on their website. The Agent has been instructed to be professional and engaling, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on {name} in the form of their summary and Linkedin details. Here's the information:"
evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Linkedin Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the user: \n\n{message}\n\n"
    user_prompt += f"Here's the latest reponse from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether the response is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",
)

In [15]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'As of now, I do not hold a patent. My focus has been on leading engineering teams and automating infrastructure, but patent submissions have not been a part of my work. If you have any specific questions about my projects or contributions, feel free to ask!'

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The agent provides a concise and appropriate answer to the user's question. The response maintains a professional tone and accurately reflects the information available in the provided context. It also offers to answer further questions, which is helpful and engaging.")

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [20]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The agent used gibberish for the response, which is not professional, and does not fit the instructions given.
